In [10]:
import numpy as np
import matplotlib.pyplot as plt
import test
%matplotlib tk

import blp_fft
# Example to plot the bases of the example lattices. Each point in the graph represent
# A chebyshev basis function

#One quadrant of a (nearly) regular hexagon
U = blp_fft.HexLattice(2)
U.plot_basis()

#One quadrant of the L_1 ball in 2d
U = blp_fft.PadLattice(10)
U.plot_basis()

#one quadrant of a rhombic dodecahedron, which is a stellation of an octahedron
U = blp_fft.BCCLattice(7)
U.plot_basis()

#one quadrant of a truncated octahedron
U = blp_fft.FCCLattice(6)
U.plot_basis()

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import test
from scipy.special import roots_legendre
import blp_fft as bf
%matplotlib tk

# Compare integration accuracy of the Padua and Hex lattices as well as Gauss-Legendre quadrature.
# Hex performs best in some middling resolution regimes, but GL is safer.
x_0 = -.2
y_0 = .3
leg_res = 400
x, w = roots_legendre(leg_res)
W = np.outer(w, w)
X, Y = np.meshgrid(x, x)

f = lambda x, y: 1/(1 + 36*(x_0 - x)**2 + 36*(y_0 - y)**2)
I = np.sum(f(X, Y)*W)

pad_pcount = []
pad_error  = []
for i in range(2, 190, 5):
    U = bf.PadLattice(i)
    U.eval_func(f)
    pad_pcount.append(U.get_point_count())
    pad_error.append(np.abs(I - U.get_integral()))
    
hex_pcount = []
hex_error  = []
for i in range(1, 40):
    U = bf.HexLattice(i)
    U.eval_func(f)
    hex_pcount.append(U.get_point_count())
    hex_error.append(np.abs(I - U.get_integral()))

leg_pcount = []
leg_error  = []
for i in range(2, 153, 10):
    x, w = roots_legendre(i)
    W = np.outer(w, w)
    X, Y = np.meshgrid(x, x)
    leg_pcount.append(i**2)
    leg_error.append(np.abs(I - np.sum(f(X, Y)*W)))
    
plt.semilogy(np.sqrt(np.array(pad_pcount)), pad_error)
plt.semilogy(np.sqrt(np.array(hex_pcount)), hex_error)
plt.semilogy(np.sqrt(np.array(leg_pcount)), leg_error)
plt.show()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import test
from scipy.special import roots_legendre
import blp_fft as bf
%matplotlib tk

# Compare integration accuracy of the BCC and FCC lattices as well as Gauss-Legendre quadrature.

x_0 = -.2
y_0 = .3
z_0 = .1
leg_res = 90
x, w = roots_legendre(leg_res)
W = np.einsum("i, j, k", w, w, w)
X, Y, Z = np.meshgrid(x, x, x)

f = lambda x, y, z: 1/(1 + 25*(x_0 - x)**2 + 25*(y_0 - y)**2 + 25*(z_0 - z)**2)
I = np.sum(f(X, Y, Z)*W)

BCC_pcount = []
BCC_error  = []
for i in range(1, 50, 1):
    print(i)
    U = bf.BCCLattice(i)
    U.eval_func(f)
    BCC_pcount.append(U.get_point_count())
    BCC_error.append(np.abs(I - U.get_integral()))
    
FCC_pcount = []
FCC_error  = []
for i in range(1, 50, 1):
    print(i)
    U = bf.FCCLattice(i)
    U.eval_func(f)
    FCC_pcount.append(U.get_point_count())
    FCC_error.append(np.abs(I - U.get_integral()))

leg_pcount = []
leg_error  = []
for i in range(2, 73, 4):
    print(i)
    x, w = roots_legendre(i)
    W = np.einsum("i,j,k", w, w, w)
    X, Y, Z = np.meshgrid(x, x, x)
    leg_pcount.append(i**3)
    leg_error.append(np.abs(I - np.sum(f(X, Y, Z)*W)))
    
plt.semilogy((np.array(BCC_pcount))**(1/3), BCC_error)
plt.semilogy((np.array(FCC_pcount))**(1/3), FCC_error)
plt.semilogy((np.array(leg_pcount))**(1/3), leg_error)
plt.show()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
2
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
66
70


In [2]:
import numpy as np
import numpy.linalg as lin
import blp_fft as bf
import matplotlib.pyplot as plt
%matplotlib tk

x_0 = .1
y_0 = .2
f  = lambda x, y: 1/(1 + 25*(x_0 - x)**2 + 25*(y_0 - y)**2)
df = lambda x, y: -50*(x - x_0)/(1 + 25*(x_0 - x)**2 + 25*(y_0 - y)**2)**2

def cheb_diff(N, f, df):
    c = lambda i: 1 + ((i == 0) or (i == N))
    D = np.zeros((N + 1, N + 1))
    x = bf.cheby_points(N + 1)
    for i in range(N + 1):
        D[i,i] = -x[i]/(2*(1 - x[i]**2)) if (i != 0) and (i != N) else ((-1)**(i != 0))*(2*N**2 + 1)/6
        for j in range(i):
            D[i, j] = (c(i)/c(j))*((-1)**(i + j))/(x[i] - x[j])
            D[j, i] = (c(j)/c(i))*((-1)**(i + j))/(x[j] - x[i])
    X, Y = np.meshgrid(x, x)
    Z = f(X, Y)
    for i in range(N + 1):
        Z[i] = D @ Z[i]
    Z -= df(X, Y)
    return Z

cheb_pcount = []
cheb_error  = []
for i in range(2, 200):
    Z = cheb_diff(i, f, df)
    Z[...] = Z**2
    Z *= 4/(i**2)
    Z[0] *= .5
    Z[-1] *= .5
    Z[:, 0] *= .5
    Z[:, -1] *= .5
    cheb_pcount.append((i + 1)**2)
    cheb_error.append(np.sqrt(np.sum(Z)))
    
hex_pcount = []
hex_error = []
for i in range(1, 50):
    print(i)
    U = bf.HexLattice(i)
    U.eval_func(f)
    U.transform()
    U.get_deriv(0)
    U.inverse_transform()
    g = [L.copy() for L in U.grids]
    U.eval_func(df)
    for i in range(len(U.grids)):
        U.grids[i] -= g[i]
        U.grids[i][...] = U.grids[i]**2
    U.apply_chebyshev_weights()
    hex_pcount.append(U.get_point_count())
    hex_error.append(np.sqrt(sum([np.sum(L) for L in U.grids])))

pad_pcount = []
pad_error = []
for i in range(2, 200, 5):
    print(i)
    U = bf.PadLattice(i)
    U.eval_func(f)
    U.transform()
    U.get_deriv(0)
    U.inverse_transform()
    g = [L.copy() for L in U.grids]
    U.eval_func(df)
    for i in range(len(U.grids)):
        U.grids[i] -= g[i]
        U.grids[i][...] = U.grids[i]**2
    U.apply_chebyshev_weights()
    pad_pcount.append(U.get_point_count())
    pad_error.append(np.sqrt(sum([np.sum(L) for L in U.grids])))
        
plt.semilogy(np.sqrt(np.array(cheb_pcount)), cheb_error)
plt.semilogy(np.sqrt(np.array(hex_pcount)), hex_error)
plt.semilogy(np.sqrt(np.array(pad_pcount)), pad_error)
plt.show()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
2
7
12
17
22
27
32
37
42
47
52
57
62
67
72
77
82
87
92
97
102
107
112
117
122
127
132
137
142
147
152
157
162
167
172
177
182
187
192
197


In [1]:
import test
import blp_fft as bf
import numpy as np

U = bf.PadLattice(15)
test.test_int_accuracy(U)

(0, 0) 4.0 4.0
(0, 1) 0.0 8.326672684688674e-17
(0, 2) -1.3333333333333333 -1.3333333333333335
(0, 3) 0.0 2.7755575615628914e-17
(0, 4) -0.26666666666666666 -0.2666666666666666
(0, 5) 0.0 4.2760933682828295e-16
(0, 6) -0.11428571428571428 -0.11428571428571385
(0, 7) 0.0 -6.277727480437707e-16
(1, 0) 0.0 4.215378046623641e-16
(1, 1) 0.0 -5.204170427930421e-18
(1, 2) 0.0 -9.280770596475918e-17
(1, 3) 0.0 1.1709383462843448e-17
(1, 4) 0.0 -2.2985086056692694e-17
(1, 5) 0.0 1.9949319973733282e-17
(1, 6) 0.0 5.637851296924623e-18
(1, 7) 0.0 -3.926737127104592e-18
(2, 0) -1.3333333333333333 -1.333333333333334
(2, 1) 0.0 -5.551115123125783e-17
(2, 2) 0.4444444444444444 0.4444444444444447
(2, 3) 0.0 0.0
(2, 4) 0.08888888888888888 0.08888888888888896
(2, 5) 0.0 -1.734723475976807e-16
(2, 6) 0.03809523809523809 0.03809523809523793
(2, 7) 0.0 2.0794867030555546e-16
(3, 0) 0.0 1.5439038936193583e-16
(3, 1) 0.0 -1.3877787807814457e-17
(3, 2) 0.0 -1.0321604682062002e-16
(3, 3) 0.0 3.903127820947816e

In [4]:
U.coefs[1]

array([[0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.5 , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ]])

In [24]:
import matplotlib.pyplot as plt
%matplotlib tk
t = np.linspace(-np.pi, np.pi, 100)
r_1 = np.cos(t)
r_2 = np.sin(t)
plt.plot(r_1*np.cos(t), r_1*np.sin(t))
plt.plot(r_2*np.cos(t), r_2*np.sin(t))
plt.plot(-r_1*np.cos(t), -r_1*np.sin(t))
plt.plot(-r_2*np.cos(t), -r_2*np.sin(t))
plt.show()

In [2]:
test.test_invertibility(U)

7.938288718631387e-16
1.061809488300179e-15


[array([[0.58374207, 0.28307268, 0.25346551, 0.62228914, 0.44599955,
         0.53398137, 0.22905578, 0.43440956],
        [0.17824966, 0.60534404, 0.98641624, 0.62786555, 0.10256071,
         0.51616633, 0.15543663, 0.76295868],
        [0.52679034, 0.03575601, 0.47406434, 0.44788915, 0.25789924,
         0.21735807, 0.47829774, 0.60821494],
        [0.03680063, 0.31923046, 0.27034548, 0.72629748, 0.46232116,
         0.42516746, 0.11934163, 0.00425909],
        [0.09176197, 0.9879996 , 0.34790996, 0.74243541, 0.09756071,
         0.20533519, 0.02364514, 0.30587048],
        [0.80549623, 0.92944698, 0.31227806, 0.87748313, 0.24381142,
         0.61005816, 0.92047699, 0.29465217],
        [0.22570837, 0.60613698, 0.5667757 , 0.96317035, 0.22866489,
         0.11493784, 0.31923338, 0.02620336],
        [0.48932298, 0.11099205, 0.56332553, 0.69921852, 0.85755754,
         0.93733673, 0.28520703, 0.52837708]]),
 array([[0.78707179, 0.54242719, 0.80514197, 0.09560315, 0.4726613 ,
         